In [62]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))

import pandas as pd
import numpy as np
import decimal
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from util.db_util import DatabaseUtility
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import rankdata

# Read .bashrc and update os.environ
bashrc_path = os.path.expanduser("~/.bashrc")

if os.path.exists(bashrc_path):
    with open(bashrc_path, "r") as f:
        for line in f:
            if line.startswith("export "):
                key_value = line.replace("export ", "").strip().split("=", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    os.environ[key] = value.strip().strip("'").strip('"')

# Initialize database connection
db_util = DatabaseUtility()

# Execute query
db_util.cursor.execute("""
    SELECT player_id, name, draft_cap, cupps_score, production_score, size_score, early_breakout, avg_fppg_nfl, total_fantasy_points_nfl FROM wr_model_data
    WHERE draft_year NOT IN (2024, 2025)
""")
rows = db_util.cursor.fetchall()
columns = [desc[0] for desc in db_util.cursor.description]
df_train = pd.DataFrame(rows, columns=columns)

# Fetch 2025 WR prospects
db_util.cursor.execute("""
    SELECT player_id, name, draft_cap, cupps_score, production_score, size_score, early_breakout, avg_fppg_nfl, total_fantasy_points_nfl FROM wr_model_data
    WHERE draft_year = 2024;
""")
rows_2025 = db_util.cursor.fetchall()
df_test = pd.DataFrame(rows_2025, columns=columns)
db_util.conn.close()

print(list(name for name in df_test['name']))

# Fill missing draft_cap

df_test_identifiers = df_test[["player_id", "name"]].copy()

# Drop unneeded columns
drop_cols = ['player_id', 
             'position', 
             'name', 
             'avg_pff_run_grade', 
             'peak_pff_run_grade', 
             'total_fantasy_points_nfl', 
             'peak_sos', 
             'peak_srs', 
             'ras', 
             'height', 
             'weight', 
             'peak_yac_per_rec', 
             'peak_tprr', 
             'avg_games_played',
             'draft_year',
             'avg_yac_per_rec',
             'avg_sos'
             ]

df_train = df_train.drop(columns=[col for col in drop_cols if col in df_train.columns])
df_test = df_test.drop(columns=[col for col in drop_cols if col in df_test.columns])

if 'avg_fppg_nfl' in df_train.columns and 'total_fantasy_points_nfl' in df_train.columns:
    df_train = df_train[~((df_train['avg_fppg_nfl'] == df_train['total_fantasy_points_nfl']) & (df_train['avg_fppg_nfl'] != 0))]

df_train = df_train.applymap(lambda x: float(x) if isinstance(x, (int, float, np.number)) or isinstance(x, decimal.Decimal) else x)
df_test = df_test.applymap(lambda x: float(x) if isinstance(x, (int, float, np.number)) or isinstance(x, decimal.Decimal) else x)

df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)

print(f"Training set size: {len(df_train)}")
print(f"Test set size: {len(df_test)}")

y_train = df_train["avg_fppg_nfl"]
y_test_actual = df_test["avg_fppg_nfl"]
X_train = df_train.drop(columns=["avg_fppg_nfl"])
X_test = df_test.drop(columns=["avg_fppg_nfl"])

# Mean of CUPPS and production_score
prod_weighted_cupps = (X_train["cupps_score"] + X_train["production_score"]) / 2

# Weight boost for high combined_score (>= 80)
sample_weights = np.log1p((X_train["cupps_score"] + X_train["production_score"]) / 2) * 2.5

xgb_general = xgb.XGBRegressor(
    max_depth=6, n_estimators=500, learning_rate=0.01,
    subsample=0.8, colsample_bytree=0.7,
    objective="reg:pseudohubererror", random_state=42
)
xgb_general.fit(X_train, y_train)

y_pred_general = xgb_general.predict(X_test)

xgb_quantile = xgb.XGBRegressor(
    max_depth=8, n_estimators=500, learning_rate=0.01,
    subsample=0.8, colsample_bytree=0.7,
    objective="reg:quantileerror", quantile_alpha=0.99,
    random_state=42
)
xgb_quantile.fit(X_train, y_train)

y_pred_quantile = xgb_quantile.predict(X_test)

meta_X = pd.DataFrame({
    "cupps_score": X_test["cupps_score"],
    "general_pred": y_pred_general,
    "quantile_pred": y_pred_quantile,
}).fillna(0)

meta_train_X = pd.DataFrame({
    "cupps_score": X_train["cupps_score"],
    "general_pred": xgb_general.predict(X_train),
    "quantile_pred": xgb_quantile.predict(X_train),
}).fillna(0)

meta_X["draft_cap"] = X_test["draft_cap"].fillna(300)
meta_train_X["draft_cap"] = X_train["draft_cap"].fillna(300)

meta_X["log_draft_cap"] = np.log1p(meta_X["draft_cap"])
meta_train_X["log_draft_cap"] = np.log1p(meta_train_X["draft_cap"])

# Add rank-based features
meta_X["cupps_rank"] = rankdata(X_test["cupps_score"], method='min') / len(X_test)
meta_train_X["cupps_rank"] = rankdata(X_train["cupps_score"], method='min') / len(X_train)

meta_X["draft_rank"] = rankdata(-X_test["draft_cap"], method='min') / len(X_test)
meta_train_X["draft_rank"] = rankdata(-X_train["draft_cap"], method='min') / len(X_train)

meta_model = xgb.XGBRegressor(
    max_depth=4, n_estimators=150, learning_rate=0.01,
    subsample=0.9, colsample_bytree=0.9,
    objective="reg:squarederror", random_state=42
)
meta_model.fit(meta_train_X, y_train)

meta_pred = meta_model.predict(meta_X)

error_general = np.abs(meta_pred - y_pred_general)
error_quantile = np.abs(meta_pred - y_pred_quantile)
elite_bonus = np.clip((meta_pred - 10) / 10, 0, 0.5)

blend_weight_quantile = np.clip(
    (error_general / (error_general + error_quantile + 1e-8)) + elite_bonus,
    0.05, 0.95
)
blend_weight_general = 1 - blend_weight_quantile

y_pred_hybrid = (y_pred_quantile * blend_weight_quantile) + (y_pred_general * blend_weight_general)

df_predictions = df_test_identifiers.copy()
df_predictions["predicted_fppg"] = y_pred_hybrid
df_predictions = df_predictions.sort_values(by="predicted_fppg", ascending=False).reset_index(drop=True)
print(df_predictions)

df_results = pd.DataFrame({
    "Actual FPPG": y_test_actual.values,
    "Predicted FPPG": y_pred_hybrid,
    "Residuals (Actual - Predicted)": y_test_actual.values - y_pred_hybrid
})

within_3 = df_results["Residuals (Actual - Predicted)"].abs() <= 3
percent_within_3 = within_3.mean() * 100
print(f"\n✅ {percent_within_3:.2f}% of players were predicted within ±3 FPPG of their actual.")

print("📊 Expected vs. Actual FPPG:")
print(df_results.sort_values(by="Actual FPPG", ascending=False).to_string(index=False))


['Tyrone Tracy Jr.', 'Jacob Cowing', "Jha'Quan Jackson", 'Ainias Smith', 'Xavier Legette', 'Malik Washington', 'Luke McCaffrey', 'Cornelius Johnson', 'Tahj Washington', 'Casey Washington', 'Jamari Thrash', 'Ricky Pearsall', 'Malachi Corley', 'Jalen McMillan', 'Rome Odunze', 'Devaughn Vele', "Ja'lynn Polk", 'Roman Wilson', 'Jermaine Burton', 'Brenden Rice', 'Johnny Wilson', 'Javon Baker', 'Tejhaun Palmer', 'Xavier Worthy', 'Anthony Gould', 'Troy Franklin', 'Marvin Harrison Jr.', 'Keon Coleman', 'Malik Nabers', 'Brian Thomas Jr.', 'Bub Means', 'Devontez Walker', 'Adonai Mitchell', 'Ladd McConkey']
Training set size: 313
Test set size: 34


/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_32925/1735297654.py:83: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_train = df_train.applymap(lambda x: float(x) if isinstance(x, (int, float, np.number)) or isinstance(x, decimal.Decimal) else x)
/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_32925/1735297654.py:84: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_test = df_test.applymap(lambda x: float(x) if isinstance(x, (int, float, np.number)) or isinstance(x, decimal.Decimal) else x)


    player_id                 name  predicted_fppg
0       17387         Malik Nabers       12.708594
1       17207  Marvin Harrison Jr.       12.205130
2       17389     Brian Thomas Jr.       12.077241
3       17487        Ladd McConkey       10.793444
4       16322          Rome Odunze       10.258060
5       17325         Keon Coleman        9.894829
6       17085        Xavier Worthy        9.730159
7       15855       Luke McCaffrey        9.151569
8       16296       Malachi Corley        8.335012
9       16390         Ja'lynn Polk        7.650890
10      17486      Adonai Mitchell        6.779666
11      16910          Javon Baker        6.335701
12      15697       Xavier Legette        5.936728
13      15553         Jacob Cowing        5.916331
14      16757      Jermaine Burton        5.670541
15      16899        Johnny Wilson        5.573112
16      16232       Ricky Pearsall        5.186178
17      17414            Bub Means        4.921267
18      17429      Devontez Wal